In [37]:
##########################################################################################################
## CODE TO RUN THROUGH LULC DATASETS AND ADD IN THE PERCENT WATER/ IRRIGATED CROPLAND ETC FOR EACH CITY ##
##########################################################################################################

import os
import numpy as np 
import earthpy as et
import pandas as pd
import pickle
from collections import Counter    #for finding the mode

# read in the city info
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT'))
CITY_COUNTRY_lat_lon = pd.read_excel('CITY_COUNTRY_lat_lon.xlsx', index_col=None)

# Define the LULC categories

value = [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220]

label = ['No Data', 'Cropland, rainfed','Cropland, irrigated or post-flooding', 
         'Mosaic cropland (>50%) / natural vegetation (tree, shrub, herbaceous cover)(<50%)'
         ,'Mosaic natural vegetation (tree, shrub, herbaceous cover) (>50%) / cropland(<50%)'
         ,'Tree cover, broadleaved, evergreen, closed to open (>15%)'
         ,'Tree cover, broadleaved, deciduous, closed to open (>15%)'
         ,'Tree cover, needleleaved, evergreen, closed to open (>15%)'
         ,'Tree cover, needleleaved, deciduous, closed to open (>15%)'
         ,'Tree cover, mixed leaf type (broadleaved and needleleaved)'
         ,'Mosaic tree and shrub (>50%) / herbaceous cover (<50%)'
         ,'Mosaic herbaceous cover (>50%) / tree and shrub (<50%)','Shrubland','Grassland','Lichens and mosses',
         'Sparse vegetation (tree, shrub, herbaceous cover) (<15%)'
         ,'Tree cover, flooded, fresh or brakish water'
         ,'Tree cover, flooded, saline water'
         ,'Shrub or herbaceous cover, flooded, fresh/saline/brakish water'
         ,'Urban areas','Bare areas','Water bodies','Permanent snow and ice']

# label 2 is more broad categories
label2 = ['No Data', 'Cropland, rainfed','Cropland, irrigated or post-flooding', 
         'Mosaic cropland / natural vegetation (tree, shrub, herbaceous cover)'
         ,'Mosaic cropland / natural vegetation (tree, shrub, herbaceous cover)'
         ,'Tree cover'
         ,'Tree cover'
         ,'Tree cover'
         ,'Tree cover'
         ,'Tree cover'
         ,'Mosaic tree and shrub / herbaceous cover'
         ,'Mosaic tree and shrub / herbaceous cover','Shrubland','Grassland','Lichens and mosses',
         'Sparse vegetation (tree, shrub, herbaceous cover) (<15%)'
         ,'Tree cover, flooded'
         ,'Tree cover, flooded'
         ,'Shrub or herbaceous cover, flooded'
         ,'Urban areas','Bare areas','Water bodies','Permanent snow and ice']

d= {'Value':value, 'Label2':label2}
LULC_key = pd.DataFrame(data = d )

In [38]:
CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['CITY_COUNTRY'] == 'HUAMBO_ANGOLA']

In [39]:
CITY_COUNTRY_lat_lon

,CITY_COUNTRY,City,Lat,Lon,City_Area,City_Bound_xmin,City_Bound_xmax,City_Bound_ymin,City_Bound_ymax,koppen_climate,...,ell_theta_2018,eccen_2018,major_ax_2019,minor_ax_2019,ell_theta_2019,eccen_2019,major_ax_2020,minor_ax_2020,ell_theta_2020,eccen_2020
0,HUAMBO_ANGOLA,Huambo,-12.77611,15.73917,73,-6000,8000,-10000,6000,Cwb,...,1.145516,0.358135,4747.419568,5089.130083,0.787862,0.360253,4822.026302,5261.610509,1.047506,0.400139


In [66]:
LULC_key[LULC_key['Value'] == 10].Label2

1    Cropland, rainfed
Name: Label2, dtype: object

In [119]:
years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013',
                  '2014','2015','2016','2017','2018','2019','2020']

# Create a dataset to be filled 
df_cols = ['CITY_COUNTRY', 'Year'
        ,'No Data', 'Cropland, rainfed','Cropland, irrigated or post-flooding' 
         ,'Mosaic cropland / natural vegetation (tree, shrub, herbaceous cover)','Tree cover'
         ,'Mosaic tree and shrub / herbaceous cover','Shrubland','Grassland','Lichens and mosses'
         ,'Sparse vegetation (tree, shrub, herbaceous cover) (<15%)','Tree cover, flooded'
         ,'Shrub or herbaceous cover, flooded' ,'Urban areas','Bare areas','Water bodies','Permanent snow and ice']

LULC_percentages_all_cities = pd.DataFrame(columns = df_cols)

# loop through all the cities
for i_count, CITY_COUNTRY in enumerate(CITY_COUNTRY_lat_lon.CITY_COUNTRY):    

    # load up LULC data for the city
    os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files', 
                      'LULC_Pickles', 'Crop_wbuffer_LULC'))

    with open('LULC_{}.pkl'.format(CITY_COUNTRY), 'rb') as f:
        LULC_df = pickle.load(f)  

    #create dataframe for the city
    LULC_percentages = pd.DataFrame(columns = df_cols)
    LULC_percentages['Year'] = years
    LULC_percentages['CITY_COUNTRY'] = CITY_COUNTRY

    for year_count, year in enumerate(years):

        lccs_class_overall_list = []

        for a in range(len(LULC_df)):
            #get list of pixel classifications
            row = LULC_df.iloc[a]
            pixel_list = []

            for i in [1,2,3,4,5]: #the numbers for the 5 pixels
                pixel_name = 'lccs_class_{}_{}'.format(i, year)
                x = row[pixel_name]
                pixel_list.append(x)
            #find the mode of the list
            counter = Counter(pixel_list)
            max_count = max(counter.values())
            mode = [k for k,v in counter.items() if v == max_count]    #use the most common pixel classification. 
                                                                #if are equal numbers, the one assigned to nearer pixel is used
            lccs_class_overall_list.append(round(mode[0], -1))         # round the values as taking less detailed classes
            lccs_class_overall_list_labels = []
            for 
            
            
            #work out the percentages of each LULC type
            LULC_number_label_df = pd.DataFrame(data = Counter(lccs_class_overall_list), index = [0])
            LULC_number_label_df_perc = 100* LULC_number_label_df/len(lccs_class_overall_list)
            #update the city table with the info for that year
            for col_count, col in enumerate(LULC_number_label_df_perc.columns):
                label2 = LULC_key[LULC_key.Value == col].Label2 
                if np.isnan(LULC_percentages[LULC_percentages['Year'] == year][label2.values[0]].values[0]) == True:
                    LULC_percentages.loc[LULC_percentages['Year']== year, label2] = LULC_number_label_df_perc[col].values[0]
                else:
                    LULC_percentages.loc[LULC_percentages['Year']== year, label2] = (LULC_percentages[LULC_percentages['Year']== year][label2.values[0]].values[0]
                                                                                     + LULC_number_label_df_perc[col].values[0])
    
    
    
# from the list calculate the percentage of each type

In [120]:
LULC_percentages

,CITY_COUNTRY,Year,No Data,"Cropland, rainfed","Cropland, irrigated or post-flooding","Mosaic cropland / natural vegetation (tree, shrub, herbaceous cover)",Tree cover,Mosaic tree and shrub / herbaceous cover,Shrubland,Grassland,Lichens and mosses,"Sparse vegetation (tree, shrub, herbaceous cover) (<15%)","Tree cover, flooded","Shrub or herbaceous cover, flooded",Urban areas,Bare areas,Water bodies,Permanent snow and ice
0,HUAMBO_ANGOLA,2000,NaN,22170.362837,NaN,1846.997251,29582.987047,28570.187304,75184.027049,NaN,NaN,NaN,NaN,NaN,2245.438512,NaN,NaN,NaN
1,HUAMBO_ANGOLA,2001,NaN,22530.587245,NaN,1732.415532,28705.501622,28402.00956,75709.494259,NaN,NaN,NaN,NaN,NaN,2519.991782,NaN,NaN,NaN
2,HUAMBO_ANGOLA,2002,NaN,22436.643403,NaN,1685.032842,28825.653316,28337.255698,75673.296104,NaN,NaN,NaN,NaN,NaN,2642.118637,NaN,NaN,NaN
3,HUAMBO_ANGOLA,2003,NaN,22436.643403,NaN,1521.279996,29098.951301,28262.701156,75563.750966,NaN,NaN,NaN,NaN,NaN,2716.673178,NaN,NaN,NaN
4,HUAMBO_ANGOLA,2004,NaN,22960.136158,NaN,1521.279996,29072.452034,28270.515087,75058.943546,NaN,NaN,NaN,NaN,NaN,2716.673178,NaN,NaN,NaN
5,HUAMBO_ANGOLA,2005,NaN,22865.69443,NaN,1294.036497,29340.749286,28146.031425,75096.99472,NaN,NaN,NaN,NaN,NaN,2856.493642,NaN,NaN,NaN
6,HUAMBO_ANGOLA,2006,NaN,23306.591299,NaN,1193.732211,28277.931479,27858.495089,75830.677728,NaN,NaN,NaN,NaN,NaN,3132.572194,NaN,NaN,NaN
7,HUAMBO_ANGOLA,2007,NaN,23481.413484,NaN,1151.400364,28143.148609,27994.092751,75613.412373,NaN,NaN,NaN,NaN,NaN,3216.532418,NaN,NaN,NaN
8,HUAMBO_ANGOLA,2008,NaN,23544.513212,NaN,1151.400364,27200.130236,27995.098439,76492.32533,NaN,NaN,NaN,NaN,NaN,3216.532418,NaN,NaN,NaN
9,HUAMBO_ANGOLA,2009,NaN,23379.026337,NaN,1151.400364,26923.790573,27780.445598,76934.151868,NaN,NaN,NaN,NaN,NaN,3431.18526,NaN,NaN,NaN


In [81]:
lccs_class_overall_list

In [101]:
LULC_percentages

,CITY_COUNTRY,Year,No Data,"Cropland, rainfed","Cropland, irrigated or post-flooding","Mosaic cropland / natural vegetation (tree, shrub, herbaceous cover)",Tree cover,Mosaic tree and shrub / herbaceous cover,Shrubland,Grassland,Lichens and mosses,"Sparse vegetation (tree, shrub, herbaceous cover) (<15%)","Tree cover, flooded","Shrub or herbaceous cover, flooded",Urban areas,Bare areas,Water bodies,Permanent snow and ice
0,HUAMBO_ANGOLA,2000,NaN,15.726817,NaN,0.18797,26.754386,11.842105,38.032581,NaN,NaN,NaN,NaN,NaN,1.879699,NaN,NaN,NaN
1,HUAMBO_ANGOLA,2001,NaN,15.914787,NaN,0.18797,26.253133,11.779449,38.471178,NaN,NaN,NaN,NaN,NaN,2.130326,NaN,NaN,NaN
2,HUAMBO_ANGOLA,2002,NaN,15.789474,NaN,0.18797,26.566416,11.716792,38.345865,NaN,NaN,NaN,NaN,NaN,2.255639,NaN,NaN,NaN
3,HUAMBO_ANGOLA,2003,NaN,15.789474,NaN,0.18797,26.879699,11.654135,38.220551,NaN,NaN,NaN,NaN,NaN,2.318296,NaN,NaN,NaN
4,HUAMBO_ANGOLA,2004,NaN,15.85213,NaN,0.18797,26.754386,11.716792,38.220551,NaN,NaN,NaN,NaN,NaN,2.318296,NaN,NaN,NaN
5,HUAMBO_ANGOLA,2005,NaN,15.601504,NaN,0.18797,27.631579,11.654135,37.907268,NaN,NaN,NaN,NaN,NaN,2.443609,NaN,NaN,NaN
6,HUAMBO_ANGOLA,2006,NaN,15.66416,NaN,0.125313,27.380952,11.591479,38.157895,NaN,NaN,NaN,NaN,NaN,2.694236,NaN,NaN,NaN
7,HUAMBO_ANGOLA,2007,NaN,15.47619,NaN,0.125313,27.568922,11.967419,37.781955,NaN,NaN,NaN,NaN,NaN,2.756892,NaN,NaN,NaN
8,HUAMBO_ANGOLA,2008,NaN,15.413534,NaN,0.125313,27.506266,12.092732,37.844612,NaN,NaN,NaN,NaN,NaN,2.756892,NaN,NaN,NaN
9,HUAMBO_ANGOLA,2009,NaN,15.413534,NaN,0.125313,27.506266,11.904762,37.844612,NaN,NaN,NaN,NaN,NaN,2.944862,NaN,NaN,NaN
